# **Recommendation Agent**

## **1. Preparing the Environment**

### **1.1 Loading the Libraries and the Data**

Steps to set up the environment for the recommendation agent include loading the necessary libraries, data and preceding agents.


In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
dir = 'D:/Master BWL HU/3. Semester/Seminar Information Systems/Seminar-Information-Systems-main'
os.chdir(dir)

from helper_functions import Helper
from agents import Preparation_Agent, Activity_Agent, Usage_Agent, Price_Agent, Load_Agent
import pandas as pd

helper = Helper()

dbfile  = "D:/Master BWL HU/3. Semester/Seminar Information Systems/Seminar-Information-Systems-main/home-assistant_Chris.db"


C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 0
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 0
}

aggregate_params = {
    'resample_param': '60T'
}
aggregate_params24_H = {
    'resample_param': '24H'
}


activity_params = {
    'active_appliances': [573,579,603, 605],
    'threshold': .15
}

time_params = {
    'features': ['hour', 'day_name']
}

activity_lag_params = {
    'features': ['activity'],
    'lags': [24, 48, 72]
}

shiftable_devices = [573,579]

device = {
    'threshold' : .15}

activity_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'activity': activity_params,
    'time': time_params,
    'activity_lag': activity_lag_params
}

usage_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'activity': activity_params,
    'aggregate_hour': aggregate_params,
    'aggregate_day': aggregate_params24_H,
    'time': time_params,
    'activity_lag': activity_lag_params,
    'shiftable_devices' : shiftable_devices,
    'device': device
}

device_params = {
    'threshold': 0.15
}

load_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'shiftable_devices': shiftable_devices, 
    'device': device_params
}

date = '2022-11-30'

### **1.2 Creating the different dataframes**

In [3]:
# calling the preparation pipeline
prep = Preparation_Agent(helper.export_sql2(dbfile))

In [4]:
# creating activity df
activity_df = prep.pipeline_activity(prep.input, activity_pipe_params)
activity_df

,activity,hour,activity_lag_24,activity_lag_48,activity_lag_72,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
last_updated,,,,,,,,,,,
2022-11-21 17:00:00,1,17,NaN,NaN,NaN,1,0,0,0,0,0
2022-11-21 18:00:00,1,18,NaN,NaN,NaN,1,0,0,0,0,0
2022-11-21 19:00:00,0,19,NaN,NaN,NaN,1,0,0,0,0,0
2022-11-21 20:00:00,1,20,NaN,NaN,NaN,1,0,0,0,0,0
2022-11-21 21:00:00,0,21,NaN,NaN,NaN,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30 13:00:00,1,13,0.0,0.0,0.0,0,0,0,0,0,1
2022-11-30 14:00:00,1,14,0.0,0.0,0.0,0,0,0,0,0,1
2022-11-30 15:00:00,0,15,0.0,0.0,0.0,0,0,0,0,0,1


In [5]:
# creating load df
load_output, load_scaled, load_df = prep.pipeline_load(prep.input, load_pipe_params)
load_df

attributes_id,573,579,603,605,573_usage,579_usage
last_updated,,,,,,
2022-11-21 17:00:00,22.990909,0.000000,0.0,0.0,1,0
2022-11-21 18:00:00,24.871795,0.320513,0.0,0.0,1,0
2022-11-21 19:00:00,0.000000,0.000000,0.0,0.0,0,0
2022-11-21 20:00:00,14.541727,1.000719,0.0,0.0,1,0
2022-11-21 21:00:00,9.292381,0.882857,0.0,0.0,0,0
...,...,...,...,...,...,...
2022-11-30 13:00:00,29.324176,3.995055,0.0,0.0,1,1
2022-11-30 14:00:00,26.513514,1.751351,0.0,0.0,1,0
2022-11-30 15:00:00,0.000000,0.000000,0.0,0.0,0,0


In [6]:
# creating usage df
usage_df = prep.pipeline_usage(prep.input, usage_pipe_params)
usage_df

,activity,573_usage,579_usage,periods_since_last_activity,periods_since_last_573_usage,periods_since_last_579_usage,hour,activity_lag_1,activity_lag_2,activity_lag_3,...,579_usage_lag_1,579_usage_lag_2,579_usage_lag_3,active_last_2_days,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
last_updated,,,,,,,,,,,,,,,,,,,,,
2022-11-21,1,1,0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
2022-11-22,0,0,0,1.0,1.0,2.0,0,1.0,0.0,0.0,...,0.0,0.0,0.0,1,0,0,0,0,1,0
2022-11-23,0,0,0,2.0,2.0,3.0,0,0.0,1.0,0.0,...,0.0,0.0,0.0,1,0,0,0,0,0,1
2022-11-24,0,0,0,3.0,3.0,4.0,0,0.0,0.0,1.0,...,0.0,0.0,0.0,0,0,0,0,1,0,0
2022-11-25,0,0,0,4.0,4.0,5.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2022-11-26,0,0,0,5.0,5.0,6.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,1,0,0,0,0
2022-11-27,0,0,0,6.0,6.0,7.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,1,0,0,0
2022-11-28,0,0,0,7.0,7.0,8.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
2022-11-29,1,0,1,8.0,8.0,9.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,1,0


In [7]:
# creating price df
PA = Price_Agent()
price_df = PA.return_day_ahead_prices(date)
price_df

2022-11-30 00:00:00+01:00    288.45
2022-11-30 01:00:00+01:00    279.18
2022-11-30 02:00:00+01:00    264.27
2022-11-30 03:00:00+01:00    260.79
2022-11-30 04:00:00+01:00    267.63
2022-11-30 05:00:00+01:00    290.00
2022-11-30 06:00:00+01:00    332.28
2022-11-30 07:00:00+01:00    434.43
2022-11-30 08:00:00+01:00    478.10
2022-11-30 09:00:00+01:00    478.60
2022-11-30 10:00:00+01:00    474.71
2022-11-30 11:00:00+01:00    482.52
2022-11-30 12:00:00+01:00    449.99
2022-11-30 13:00:00+01:00    450.47
2022-11-30 14:00:00+01:00    464.24
2022-11-30 15:00:00+01:00    458.76
2022-11-30 16:00:00+01:00    478.40
2022-11-30 17:00:00+01:00    503.55
2022-11-30 18:00:00+01:00    495.20
2022-11-30 19:00:00+01:00    465.14
2022-11-30 20:00:00+01:00    413.90
2022-11-30 21:00:00+01:00    344.20
2022-11-30 22:00:00+01:00    316.86
2022-11-30 23:00:00+01:00    277.18
2022-12-01 00:00:00+01:00    292.06
2022-12-01 01:00:00+01:00    291.43
2022-12-01 02:00:00+01:00    284.76
2022-12-01 03:00:00+01:00   

## **2. Creating the original Recommendation Agent**

In [8]:
class Recommendation_Agent:
    def __init__(
        self, activity_input, usage_input, load_input, price_input, shiftable_devices):
        self.activity_input = activity_input
        self.usage_input = usage_input
        self.load_input = load_input
        self.price_input = price_input
        self.shiftable_devices = shiftable_devices
        self.Activity_Agent = Activity_Agent(activity_input)
        # create dictionary with Usage_Agent for each device
        self.Usage_Agent = {
            name: Usage_Agent(usage_input, name) for name in shiftable_devices
        }
        self.Load_Agent = Load_Agent(load_input)
        self.Price_Agent = Price_Agent()

### **2.1 Calculation Costs**

In [9]:
# calculating costs
# -------------------------------------------------------------------------------------------
def electricity_prices_from_start_time(self, date):
    import pandas as pd

    prices_48 = self.Price_Agent.return_day_ahead_prices(date)
    prices_from_start_time = pd.DataFrame()
    for i in range(24):
        prices_from_start_time["Price_at_H+" + str(i)] = prices_48.shift(-i)
    # delete last 24 hours
    prices_from_start_time = prices_from_start_time[:-24]
    return prices_from_start_time
# add to recommendation agent
setattr(Recommendation_Agent, 'electricity_prices_from_start_time', electricity_prices_from_start_time)
del electricity_prices_from_start_time

In [11]:
date = '2022-11-30'
Recommendation_Agent_i = Recommendation_Agent(activity_df, usage_df, load_df, price_df, shiftable_devices) 
prices_from_start_time = Recommendation_Agent_i.electricity_prices_from_start_time(date)
prices_from_start_time

,Price_at_H+0,Price_at_H+1,Price_at_H+2,Price_at_H+3,Price_at_H+4,Price_at_H+5,Price_at_H+6,Price_at_H+7,Price_at_H+8,Price_at_H+9,...,Price_at_H+14,Price_at_H+15,Price_at_H+16,Price_at_H+17,Price_at_H+18,Price_at_H+19,Price_at_H+20,Price_at_H+21,Price_at_H+22,Price_at_H+23
2022-11-30 00:00:00+01:00,288.45,279.18,264.27,260.79,267.63,290.00,332.28,434.43,478.10,478.60,...,464.24,458.76,478.40,503.55,495.20,465.14,413.90,344.20,316.86,277.18
2022-11-30 01:00:00+01:00,279.18,264.27,260.79,267.63,290.00,332.28,434.43,478.10,478.60,474.71,...,458.76,478.40,503.55,495.20,465.14,413.90,344.20,316.86,277.18,292.06
2022-11-30 02:00:00+01:00,264.27,260.79,267.63,290.00,332.28,434.43,478.10,478.60,474.71,482.52,...,478.40,503.55,495.20,465.14,413.90,344.20,316.86,277.18,292.06,291.43
2022-11-30 03:00:00+01:00,260.79,267.63,290.00,332.28,434.43,478.10,478.60,474.71,482.52,449.99,...,503.55,495.20,465.14,413.90,344.20,316.86,277.18,292.06,291.43,284.76
2022-11-30 04:00:00+01:00,267.63,290.00,332.28,434.43,478.10,478.60,474.71,482.52,449.99,450.47,...,495.20,465.14,413.90,344.20,316.86,277.18,292.06,291.43,284.76,274.80
2022-11-30 05:00:00+01:00,290.00,332.28,434.43,478.10,478.60,474.71,482.52,449.99,450.47,464.24,...,465.14,413.90,344.20,316.86,277.18,292.06,291.43,284.76,274.80,271.91
2022-11-30 06:00:00+01:00,332.28,434.43,478.10,478.60,474.71,482.52,449.99,450.47,464.24,458.76,...,413.90,344.20,316.86,277.18,292.06,291.43,284.76,274.80,271.91,283.49
2022-11-30 07:00:00+01:00,434.43,478.10,478.60,474.71,482.52,449.99,450.47,464.24,458.76,478.40,...,344.20,316.86,277.18,292.06,291.43,284.76,274.80,271.91,283.49,353.18
2022-11-30 08:00:00+01:00,478.10,478.60,474.71,482.52,449.99,450.47,464.24,458.76,478.40,503.55,...,316.86,277.18,292.06,291.43,284.76,274.80,271.91,283.49,353.18,450.84
2022-11-30 09:00:00+01:00,478.60,474.71,482.52,449.99,450.47,464.24,458.76,478.40,503.55,495.20,...,277.18,292.06,291.43,284.76,274.80,271.91,283.49,353.18,450.84,463.55


In [12]:
def cost_by_starting_time(self, date, device, evaluation=False):
    import numpy as np
    import pandas as pd

    # get electriciy prices following every device starting hour with previously defined function
    prices = self.electricity_prices_from_start_time(date)
    # build up table with typical load profile repeated for every hour (see Load_Agent)
    if not evaluation:
        device_load = self.Load_Agent.pipeline(
            self.load_input, date, self.shiftable_devices
        ).loc[device]
    else:
        # get device load for one date
        device_load = evaluation["load"][date].loc[device]
    device_load = pd.concat([device_load] * 24, axis=1)
    # multiply both tables and aggregate costs for each starting hour
    costs = np.array(prices) * np.array(device_load)
    costs = np.sum(costs, axis=0)
    # return an array of size 24 containing the total cost at each staring hour.
    return costs

# add to recommendation agent
setattr(Recommendation_Agent, 'cost_by_starting_time', cost_by_starting_time)
del cost_by_starting_time

In [15]:
Recommendation_Agent_i = Recommendation_Agent(activity_df, usage_df, load_df, price_df, shiftable_devices) 
device = 573

costs = Recommendation_Agent_i.cost_by_starting_time(date, device)
costs

array([152.89108453, 146.21068902, 141.5662791 , 142.75927799,
       151.07130793, 169.06674279, 209.69627827, 247.41570926,
       258.13461475, 257.09812528, 258.48051968, 250.69123614,
       242.96032677, 247.17392323, 248.87625499, 253.39394346,
       265.63137472, 269.23418376, 258.26842129, 235.74551053,
       202.60310144, 177.59721619, 159.17247395, 153.99581763])

# Current state

# Current state

### **2.2 Creating Recommendation**

In [ ]:
# creating recommendations
# -------------------------------------------------------------------------------------------
def recommend_by_device(
    self,
    date,
    device,
    activity_prob_threshold,
    usage_prob_threshold,
    evaluation=False,
    weather_sel=False
):
    import numpy as np

    # add split params as input
    # IN PARTICULAR --> Specify date to start training
    split_params = {
        "train_start": "2013-11-01",
        "test_delta": {"days": 1, "seconds": -1},
        "target": "activity",
    }
    # compute costs by launching time:
    costs = self.cost_by_starting_time(date, device, evaluation=evaluation)
    # compute activity probabilities
    if not evaluation:
        if weather_sel:
            activity_probs = self.Activity_Agent.pipeline(self.activity_input, date, self.model_type, split_params, weather_sel=True)
        else:
            activity_probs = self.Activity_Agent.pipeline(self.activity_input, date, self.model_type, split_params)
    else:
        # get activity probs for date
        activity_probs = evaluation["activity"][date]

    # set values above threshold to 1. Values below to Inf
    # (vector will be multiplied by costs, so that hours of little activity likelihood get cost = Inf)
    activity_probs = np.where(activity_probs >= activity_prob_threshold, 1, float("Inf"))

    # add a flag in case all hours have likelihood smaller than threshold
    no_recommend_flag_activity = 0
    if np.min(activity_probs) == float("Inf"):
        no_recommend_flag_activity = 1

    # compute cheapest hour from likely ones
    best_hour = np.argmin(np.array(costs) * np.array(activity_probs))

    # compute likelihood of usage:
    if not evaluation:
        usage_prob = self.Usage_Agent[device].pipeline(self.usage_input, date, self.model_type, split_params["train_start"])
    else:
        # get usage probs
        name = ("usage_" + device.replace(" ", "_").replace("(", "").replace(")", "").lower())
        usage_prob = evaluation[name][date]


    no_recommend_flag_usage = 0
    if usage_prob < usage_prob_threshold:
        no_recommend_flag_usage = 1

    return {
        "recommendation_date": [date],
        "device": [device],
        "best_launch_hour": [best_hour],
        "no_recommend_flag_activity": [no_recommend_flag_activity],
        "no_recommend_flag_usage": [no_recommend_flag_usage],
        "recommendation": [
            best_hour
            if (no_recommend_flag_activity == 0 and no_recommend_flag_usage == 0)
            else np.nan
        ],
    }

# add to recommendation agent
setattr(Recommendation_Agent, 'recommend_by_device', recommend_by_device)
del recommend_by_device

In [ ]:
output = Recommendation_Agent.recommend_by_device(self,
                             date,
                             device,
                             activity_prob_threshold,
                             usage_prob_threshold,
                             evaluation=False,
                             weather_sel=False)


In [ ]:
# The Original Recommendation Agent
# ===============================================================================================
class Recommendation_Agent:
    def __init__(
        self, activity_input, usage_input, load_input, price_input, shiftable_devices):
        self.activity_input = activity_input
        self.usage_input = usage_input
        self.load_input = load_input
        self.price_input = price_input
        self.shiftable_devices = shiftable_devices
        self.Activity_Agent = Activity_Agent(activity_input)
        # create dicionnary with Usage_Agent for each device
        self.Usage_Agent = {
            name: Usage_Agent(usage_input, name) for name in shiftable_devices
        }
        self.Load_Agent = Load_Agent(load_input)
        self.Price_Agent = Price_Agent()

    # calculating costs
    # -------------------------------------------------------------------------------------------
    def electricity_prices_from_start_time(self, date):
        import pandas as pd

        prices_48 = self.Price_Agent.return_day_ahead_prices(date)
        prices_from_start_time = pd.DataFrame()
        for i in range(24):
            prices_from_start_time["Price_at_H+" + str(i)] = prices_48.shift(-i)
        # delete last 24 hours
        prices_from_start_time = prices_from_start_time[:-24]
        return prices_from_start_time

    def cost_by_starting_time(self, date, device, evaluation=False):
        import numpy as np
        import pandas as pd

        # get electriciy prices following every device starting hour with previously defined function
        prices = self.electricity_prices_from_start_time(date)
        # build up table with typical load profile repeated for every hour (see Load_Agent)
        if not evaluation:
            device_load = self.Load_Agent.pipeline(
                self.load_input, date, self.shiftable_devices
            ).loc[device]
        else:
            # get device load for one date
            device_load = evaluation["load"][date].loc[device]
        device_load = pd.concat([device_load] * 24, axis=1)
        # multiply both tables and aggregate costs for each starting hour
        costs = np.array(prices) * np.array(device_load)
        costs = np.sum(costs, axis=0)
        # return an array of size 24 containing the total cost at each staring hour.
        return costs

    # creating recommendations
    # -------------------------------------------------------------------------------------------
    def recommend_by_device(
        self,
        date,
        device,
        activity_prob_threshold,
        usage_prob_threshold,
        evaluation=False,
        weather_sel=False
    ):
        import numpy as np

        # add split params as input
        # IN PARTICULAR --> Specify date to start training
        split_params = {
            "train_start": "2013-11-01",
            "test_delta": {"days": 1, "seconds": -1},
            "target": "activity",
        }
        # compute costs by launching time:
        costs = self.cost_by_starting_time(date, device, evaluation=evaluation)
        # compute activity probabilities
        if not evaluation:
            if weather_sel:
                activity_probs = self.Activity_Agent.pipeline(self.activity_input, date, self.model_type, split_params, weather_sel=True)
            else:
                activity_probs = self.Activity_Agent.pipeline(self.activity_input, date, self.model_type, split_params)
        else:
            # get activity probs for date
            activity_probs = evaluation["activity"][date]

        # set values above threshold to 1. Values below to Inf
        # (vector will be multiplied by costs, so that hours of little activity likelihood get cost = Inf)
        activity_probs = np.where(activity_probs >= activity_prob_threshold, 1, float("Inf"))

        # add a flag in case all hours have likelihood smaller than threshold
        no_recommend_flag_activity = 0
        if np.min(activity_probs) == float("Inf"):
            no_recommend_flag_activity = 1

        # compute cheapest hour from likely ones
        best_hour = np.argmin(np.array(costs) * np.array(activity_probs))

        # compute likelihood of usage:
        if not evaluation:
            usage_prob = self.Usage_Agent[device].pipeline(self.usage_input, date, self.model_type, split_params["train_start"])
        else:
            # get usage probs
            name = ("usage_" + device.replace(" ", "_").replace("(", "").replace(")", "").lower())
            usage_prob = evaluation[name][date]


        no_recommend_flag_usage = 0
        if usage_prob < usage_prob_threshold:
            no_recommend_flag_usage = 1

        return {
            "recommendation_date": [date],
            "device": [device],
            "best_launch_hour": [best_hour],
            "no_recommend_flag_activity": [no_recommend_flag_activity],
            "no_recommend_flag_usage": [no_recommend_flag_usage],
            "recommendation": [
                best_hour
                if (no_recommend_flag_activity == 0 and no_recommend_flag_usage == 0)
                else np.nan
            ],
        }

    # visualize recommendation_by device
    def visualize_recommendation_by_device(self, dict):
        recommendation_date = str(dict['recommendation_date'][0])
        recommendation_date = datetime.strptime(recommendation_date, '%Y-%m-%d')
        recommendation_date = recommendation_date.strftime(format = "%d.%m.%Y %H:%M")
        device = dict['device'][0]
        best_launch_hour = dict['best_launch_hour'][0]
        if (dict['no_recommend_flag_activity'][0]== 0 and dict['no_recommend_flag_usage'][0]==0) == True:
            return print('You have one recommendation for the following device: ' + device + '\nPlease use it on ' + recommendation_date[0:10] + ' at '+ recommendation_date[11:]+'.')


    # vizualizing the recommendations
    # -------------------------------------------------------------------------------------------
    def recommendations_on_date_range(
        self, date_range, activity_prob_threshold=0.6, usage_prob_threshold=0.5
    ):
        import pandas as pd

        recommendations = []
        for date in date_range:
            recommendations.append(self.pipeline(date, activity_prob_threshold, usage_prob_threshold))
            output = pd.concat(recommendations)
        return output

    def visualize_recommendations_on_date_range(self, recs):
        import plotly.express as px
        import plotly.graph_objects as go

        fig = go.Figure()

        for device in recs["device"].unique():
            plot_device = recs[recs["device"] == device]
            fig.add_trace(
                go.Scatter(
                    x=plot_device["recommendation_date"],
                    y=plot_device["recommendation"],
                    mode="lines",
                    name=device,
                )
            )
        fig.show()

    def histogram_recommendation_hour(self, recs):
        import seaborn as sns

        ax = sns.displot(recs, x="recommendation", binwidth=1)
        ax.set(xlabel="Hour of Recommendation", ylabel="counts")

    # pipeline function: create recommendations
    # -------------------------------------------------------------------------------------------
    def pipeline(self, date, activity_prob_threshold, usage_prob_threshold, evaluation=False, weather_sel=False):
        import pandas as pd

        recommendations_by_device = self.recommend_by_device(
            date,
            self.shiftable_devices[0],
            activity_prob_threshold,
            usage_prob_threshold,
            evaluation=evaluation,
        )
        recommendations_table = pd.DataFrame.from_dict(recommendations_by_device)

        for device in self.shiftable_devices[1:]:
            if weather_sel:
                recommendations_by_device = self.recommend_by_device(
                    date,
                    device,
                    activity_prob_threshold,
                    usage_prob_threshold,
                    evaluation=evaluation,
                    weather_sel=True
                )
            else:
                recommendations_by_device = self.recommend_by_device(
                    date,
                    device,
                    activity_prob_threshold,
                    usage_prob_threshold,
                    evaluation=evaluation,
                )
            recommendations_table = recommendations_table.append(
                pd.DataFrame.from_dict(recommendations_by_device)
            )
        return recommendations_table

    def visualize_recommendation(self, recommendations_table):

        for i in range(len(recommendations_table)):
            date_and_time = recommendations_table.recommendation_date.iloc[i] + ':' + str(recommendations_table.best_launch_hour.iloc[i])

            date_and_time = datetime.strptime(date_and_time, '%Y-%m-%d:%H')

            date_and_time_show = date_and_time.strftime(format = "%d.%m.%Y %H:%M")
            date_and_time_price = date_and_time.strftime(format = "%Y-%m-%d %H:%M:%S")
            price = price.filter(like=date_and_time_price, axis=0)['Price_at_H+0'].iloc[0]
            output = print('You have a recommendation for the following device: ' + recommendations_table.device.iloc[i]+ '\n\nPlease use the device on the ' + date_and_time_show[0:10] + ' at ' + date_and_time_show[11:] + ' oclock because it costs you only ' + str(price) + ' €.\n')
            if (recommendations_table.no_recommend_flag_activity.iloc[i]==0 and recommendations_table.no_recommend_flag_usage.iloc[i]==0) == True:
                return output
            else:
                return


In [ ]:
# X_Recommendation Agent
# ===============================================================================================
class X_Recommendation_Agent:
    def __init__(
        self, activity_input, usage_input, load_input, price_input, shiftable_devices, best_hour = None, model_type = 'logit'):
        self.activity_input = activity_input
        self.usage_input = usage_input
        self.load_input = load_input
        self.price_input = price_input
        self.shiftable_devices = shiftable_devices
        self.model_type = model_type
        self.Activity_Agent = Activity_Agent(activity_input)
        # create dicionnary with Usage_Agent for each device
        self.Usage_Agent = {
            name: Usage_Agent(usage_input, name) for name in shiftable_devices
        }
        self.Load_Agent = Load_Agent(load_input)
        self.Price_Agent = Price_Agent(price_input)
        self.best_hour = best_hour

    # calculating costs
    # -------------------------------------------------------------------------------------------
    def electricity_prices_from_start_time(self, date):
        import pandas as pd

        prices_48 = self.Price_Agent.return_day_ahead_prices(date)
        prices_from_start_time = pd.DataFrame()
        for i in range(24):
            prices_from_start_time["Price_at_H+" + str(i)] = prices_48.shift(-i)
        # delete last 24 hours
        prices_from_start_time = prices_from_start_time[:-24]
        return prices_from_start_time

    def cost_by_starting_time(self, date, device, evaluation=False):
        import numpy as np
        import pandas as pd

        # get electriciy prices following every device starting hour with previously defined function
        prices = self.electricity_prices_from_start_time(date)
        # build up table with typical load profile repeated for every hour (see Load_Agent)
        if not evaluation:
            device_load = self.Load_Agent.pipeline(
                self.load_input, date, self.shiftable_devices
            ).loc[device]
        else:
            # get device load for one date
            device_load = evaluation["load"][date].loc[device]
        device_load = pd.concat([device_load] * 24, axis=1)
        # multiply both tables and aggregate costs for each starting hour
        costs = np.array(prices) * np.array(device_load)
        costs = np.sum(costs, axis=0)
        # return an array of size 24 containing the total cost at each staring hour.
        return costs

    # creating recommendations
    # -------------------------------------------------------------------------------------------
    def recommend_by_device(
        self,
        date,
        device,
        activity_prob_threshold,
        usage_prob_threshold,
        evaluation=False,
        weather_sel=False
    ):
        import numpy as np

        # add split params as input
        # IN PARTICULAR --> Specify date to start training
        split_params = {
            "train_start": "2013-11-01",
            "test_delta": {"days": 1, "seconds": -1},
            "target": "activity",
        }
        # compute costs by launching time:
        costs = self.cost_by_starting_time(date, device, evaluation=evaluation)

        X_train_activity = None
        X_test_activity = None
        model_activity = None
        model_usage = None

        # compute activity probabilities
        if not evaluation:
            if weather_sel:
                activity_probs, X_train_activity, X_test_activity, model_activity = self.Activity_Agent.pipeline_xai(
                    self.activity_input, date, self.model_type, split_params, weather_sel=True)
            else:
                activity_probs, X_train_activity, X_test_activity, model_activity = self.Activity_Agent.pipeline_xai(
                    self.activity_input, date, self.model_type, split_params, weather_sel=False)
        else:
            # get activity probs for date
            activity_probs = evaluation["activity"][date]

        # set values above threshold to 1. Values below to Inf
        # (vector will be multiplied by costs, so that hours of little activity likelihood get cost = Inf)
        activity_probs = np.where(activity_probs >= activity_prob_threshold, 1, float("Inf"))

        # add a flag in case all hours have likelihood smaller than threshold
        no_recommend_flag_activity = 0
        if np.min(activity_probs) == float("Inf"):
            no_recommend_flag_activity = 1

        # compute cheapest hour from likely ones
        self.best_hour = np.argmin(np.array(costs) * np.array(activity_probs))

        # compute likelihood of usage:
        if not evaluation:
            if weather_sel:
                usage_prob, X_train_usage, X_test_usage, model_usage = self.Usage_Agent[device].pipeline_xai(
                    self.usage_input, date,self.model_type, split_params["train_start"], weather_sel=True)
            else:
                usage_prob, X_train_usage, X_test_usage, model_usage = self.Usage_Agent[device].pipeline_xai(
                self.usage_input, date,self.model_type, split_params["train_start"], weather_sel=False)
        else:
            # get usage probs
            name = ("usage_" + device.replace(" ", "_").replace("(", "").replace(")", "").lower())
            usage_prob = evaluation[name][date]


        no_recommend_flag_usage = 0
        if usage_prob < usage_prob_threshold:
            no_recommend_flag_usage = 1

        self.Explainability_Agent = Explainability_Agent(model_activity, X_train_activity, X_test_activity, self.best_hour, model_usage,
        X_train_usage, X_test_usage, model_type=self.model_type)

        explain = Explainability_Agent(model_activity, X_train_activity, X_test_activity,
                                       self.best_hour,model_usage,X_train_usage, X_test_usage,
                                       model_type= self.model_type)
        feature_importance_activity, feature_importance_usage, explainer_activity, explainer_usage, shap_values, shap_values_usage, X_test_activity, X_test_usage = explain.feature_importance()


        return {
            "recommendation_date": [date],
            "device": [device],
            "best_launch_hour": [self.best_hour],
            "no_recommend_flag_activity": [no_recommend_flag_activity],
            "no_recommend_flag_usage": [no_recommend_flag_usage],
            "recommendation": [
                self.best_hour
                if (no_recommend_flag_activity == 0 and no_recommend_flag_usage == 0)
                else np.nan
            ],
            "feature_importance_activity": [feature_importance_activity],
            "feature_importance_usage": [feature_importance_usage],
            "explainer_activity": [explainer_activity],
            "explainer_usage": [explainer_usage],
            "shap_values": [shap_values],
            "shap_values_usage": [shap_values_usage],
            "X_test_activity": [X_test_activity],
            "X_test_usage": [X_test_usage],
        }

    # visualize recommendation_by device
    def visualize_recommendation_by_device(self, dict):
        recommendation_date = str(dict['recommendation_date'][0])
        recommendation_date = datetime.strptime(recommendation_date, '%Y-%m-%d')
        recommendation_date = recommendation_date.strftime(format = "%d.%m.%Y %H:%M")
        device = dict['device'][0]
        best_launch_hour = dict['best_launch_hour'][0]
        if (dict['no_recommend_flag_activity'][0]== 0 and dict['no_recommend_flag_usage'][0]==0) == True:
            return print('You have one recommendation for the following device: ' + device + '\nPlease use it on ' + recommendation_date[0:10] + ' at '+ recommendation_date[11:]+'.')



    # vizualizing the recommendations
    # -------------------------------------------------------------------------------------------
    def recommendations_on_date_range(
        self, date_range, activity_prob_threshold=0.6, usage_prob_threshold=0.5
    ):
        import pandas as pd

        recommendations = []
        for date in date_range:
            recommendations.append(self.pipeline(date, activity_prob_threshold, usage_prob_threshold))
            output = pd.concat(recommendations)
        return output

    def visualize_recommendations_on_date_range(self, recs):
        import plotly.express as px
        import plotly.graph_objects as go

        fig = go.Figure()

        for device in recs["device"].unique():
            plot_device = recs[recs["device"] == device]
            fig.add_trace(
                go.Scatter(
                    x=plot_device["recommendation_date"],
                    y=plot_device["recommendation"],
                    mode="lines",
                    name=device,
                )
            )
        fig.show()

    def histogram_recommendation_hour(self, recs):
        import seaborn as sns

        ax = sns.displot(recs, x="recommendation", binwidth=1)
        ax.set(xlabel="Hour of Recommendation", ylabel="counts")

    # pipeline function: create recommendations
    # -------------------------------------------------------------------------------------------
    def pipeline(self, date, activity_prob_threshold, usage_prob_threshold, evaluation=False, weather_sel=False):
        import pandas as pd

        recommendations_by_device = self.recommend_by_device(
            date,
            self.shiftable_devices[0],
            activity_prob_threshold,
            usage_prob_threshold,
            evaluation=evaluation,
        )
        recommendations_table = pd.DataFrame.from_dict(recommendations_by_device)

        for device in self.shiftable_devices[1:]:
            if weather_sel:
                recommendations_by_device = self.recommend_by_device(
                    date,
                    device,
                    activity_prob_threshold,
                    usage_prob_threshold,
                    evaluation=evaluation,
                    weather_sel=True
                )
            else:
                recommendations_by_device = self.recommend_by_device(
                    date,
                    device,
                    activity_prob_threshold,
                    usage_prob_threshold,
                    evaluation=evaluation,
                )
            recommendations_table = recommendations_table.append(
                pd.DataFrame.from_dict(recommendations_by_device)
            )
        return recommendations_table

    def visualize_recommendation(self, recommendations_table, price, diagnostics=False):
        self.diagnostics = diagnostics

        for r in range(len(recommendations_table)):
            if (recommendations_table.no_recommend_flag_activity.iloc[r] == 0 and
                recommendations_table.no_recommend_flag_usage.iloc[r] == 0) == True:

                recommendations = True
            else:
                recommendations = False

        if recommendations == True:

            feature_importance_activity = recommendations_table['feature_importance_activity'].iloc[0]
            date = recommendations_table.recommendation_date.iloc[0]
            best_hour = recommendations_table.best_launch_hour.iloc[0]
            explaination_activity = self.Explainability_Agent.explanation_from_feature_importance_activity(feature_importance_activity, date=date , best_hour=best_hour, diagnostics=self.diagnostics)

            output = []
            explaination_usage = []
            for i in range(len(recommendations_table)):

                if (recommendations_table.no_recommend_flag_activity.iloc[i] == 0 and
                recommendations_table.no_recommend_flag_usage.iloc[i] == 0) == True:

                    date_and_time = recommendations_table.recommendation_date.iloc[i] + ':' + str(recommendations_table.best_launch_hour.iloc[i])

                    date_and_time =  datetime.strptime(date_and_time, '%Y-%m-%d:%H')

                    date_and_time_show = date_and_time.strftime(format = "%d.%m.%Y %H:%M")
                    date_and_time_price = date_and_time.strftime(format = "%Y-%m-%d %H:%M:%S")

                    price_rec = price.filter(like=date_and_time_price, axis=0)['Price_at_H+0'].iloc[0]
                    price_mean = price['Price_at_H+0'].sum() / 24
                    price_dif = price_rec / price_mean
                    price_savings_percentage = round((1 - price_dif) * 100, 2)

                    output = print('You have a recommendation for the following device: ' + recommendations_table.device.iloc[i] + '\n\nPlease use the device on the ' + date_and_time_show[0:10] + ' at ' + date_and_time_show[11:] + " o'clock because it saves you " + str(price_savings_percentage) + ' % of costs compared to the mean of the day.\n')
                    feature_importance_usage_device = recommendations_table['feature_importance_usage'].iloc[i]
                    explaination_usage = self.Explainability_Agent.explanation_from_feature_importance_usage(feature_importance_usage_device, date=date, diagnostics=self.diagnostics)
                    print(explaination_usage)


                    if self.diagnostics == True:
                        print('Vizualizations for further insights into our predictions: ')
                        explainer_usage = recommendations_table['explainer_usage'].iloc[i]
                        shap_values_usage = recommendations_table['shap_values_usage'].iloc[i]
                        X_test_usage = recommendations_table['X_test_usage'].iloc[i]
                        shap_plot_usage = shap.force_plot(explainer_usage.expected_value[1], shap_values_usage[1], X_test_usage)
                        display(shap_plot_usage)

                else:
                    print('There is no recommendation for the device ' + recommendations_table.device.iloc[i] + ' .')

            print(explaination_activity)

            if self.diagnostics == True:
                print('Vizualizations for further insights into our predictions: ')
                explainer_activity = recommendations_table[recommendations_table['device']=='Tumble Dryer']['explainer_activity'][0]
                shap_values = recommendations_table[recommendations_table['device']=='Tumble Dryer']['shap_values'][0]
                X_test_activity = recommendations_table[recommendations_table['device']=='Tumble Dryer']['X_test_activity'][0]
                shap_plot_activity = shap.force_plot(explainer_activity.expected_value[1], shap_values[1], X_test_activity.iloc[best_hour, :])
                display(shap_plot_activity)

            if self.diagnostics == False:
                print('For detailed information switch on the diagnostics parameter.')
            return

        else:
            print('There are no recommendations for today.')
            return None

In [ ]:
recommend = X_Recommendation_Agent(activity_df, usage_df, load_df, price_df, shiftable_devices, model_type='random forest')
price = recommend.electricity_prices_from_start_time("2014-08-21")
table= recommend.pipeline(date = "2014-08-21", activity_prob_threshold = 0.4,  usage_prob_threshold = 0.7, evaluation=False, weather_sel=False)
table